In [1]:
import pandas as pd
import glob
from google.cloud import bigquery
from google.oauth2 import service_account
import pandas as pd

In [2]:
# 파이썬 구글 빅쿼리 연동 코드
# json 파일

key_path = glob.glob("./*.json")[0]
credentials = service_account.Credentials.from_service_account_file(key_path)
client = bigquery.Client(credentials = credentials, 
                         project = credentials.project_id)

In [3]:
def sql_to_dataframe(sql:str) -> pd.DataFrame:
    """
    Args:
        sql (str): sql for extraction

    Returns:
        pd.DataFrame: extract data with sql
    """
    query_job = client.query(sql)
    df = query_job.to_dataframe()
    return df

### 전환수

In [8]:
sql = """
SELECT
  event_name,
  COUNT(event_name) AS conversion
FROM `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_*`
WHERE _table_suffix BETWEEN "20201110" AND "20201206"
GROUP BY event_name
ORDER BY conversion DESC
"""

df = sql_to_dataframe(sql)
df

,event_name,conversion
0,page_view,440286
1,user_engagement,392661
2,scroll,169102
3,view_item,144134
4,session_start,102342
5,first_visit,69380
6,view_promotion,62199
7,add_to_cart,15488
8,begin_checkout,11825
9,view_search_results,8744


### 총사용자

In [9]:
sql = """
SELECT
  event_name,
  COUNT(DISTINCT user_pseudo_id) AS user
FROM `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_*`
WHERE _table_suffix BETWEEN "20201110" AND "20201206"
AND event_name IN("page_view","view_item","first_visit","predicted_top_spenders","add_to_cart","view_cart","begin_checkout","purchase")
GROUP BY event_name
ORDER BY user DESC
"""

df = sql_to_dataframe(sql)
df

,event_name,user
0,page_view,76272
1,first_visit,69344
2,view_item,21197
3,begin_checkout,4231
4,add_to_cart,3942
5,purchase,1681


### 총수익

In [10]:
sql = """
SELECT
  event_name,
  SUM((int_value + float_value + double_value)) AS value
FROM(
  SELECT
    event_name,
    CASE
      WHEN int_value IS NULL THEN 0
      ELSE int_value
    END AS int_value,
    CASE 
      WHEN float_value IS NULL THEN 0
      ELSE float_value
    END AS float_value,
    CASE
      WHEN double_value IS NULL THEN 0
      ELSE double_value
    END AS double_value
  FROM(
    SELECT
      event_name,
      (SELECT value.int_value FROM UNNEST(event_params) WHERE key = "value") AS int_value,
      (SELECT value.float_value FROM UNNEST(event_params) WHERE key = "value") AS float_value,
      (SELECT value.double_value FROM UNNEST(event_params) WHERE key = "value") AS double_value
    FROM `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_*`
    WHERE _table_suffix BETWEEN "20201110" AND "20201206"
  )
  )
  GROUP BY event_name
  ORDER BY value DESC"""

df = sql_to_dataframe(sql)
df

,event_name,value
0,purchase,150484.93
1,session_start,0.00
2,page_view,0.00
3,view_item,0.00
4,add_shipping_info,0.00
5,view_search_results,0.00
6,first_visit,0.00
7,scroll,0.00
8,user_engagement,0.00
9,add_payment_info,0.00
